In [1]:
import pandas as pd
import numpy as np
import time
import gc

In [2]:
path = '/home/kai/data/kaggle/talkingdata/data/'
train = pd.read_csv(path + 'train_cleaned_final.csv')
test = pd.read_csv(path + 'test_cleaned_final.csv')

In [3]:
load_path = '/home/kai/data/kaggle/talkingdata/wl/data/compensate/'
comp_index = np.load(load_path+'all_compen.npy')
df_comp = train.iloc[comp_index]

# evaluate channel, see if in intersection

In [4]:
# pool = set(test.channel.value_counts().index).intersection(set(train.iloc[index['day7']+index['day8']+index['day9']].copy().channel.value_counts().index))

# f = lambda x: x in pool
# day7_ori= train.iloc[index['day7']].copy()
# day7 = day7_ori[day7_ori.channel.apply(f)]

# day8_ori= train.iloc[index['day8']].copy()
# day8 = day8_ori[day8_ori.channel.apply(f)]

# day9_ori= train.iloc[index['day9']].copy()
# day9 = day9_ori[day9_ori.channel.apply(f)]

# print(len(day7_ori))
# print(len(day7))
# print(len(day8_ori))
# print(len(day8))
# print(len(day9_ori))
# print(len(day9))

# Feature Cols

In [5]:
target = 'is_attributed'
feature_count =  [
                    'ip_day_hour_count',
                    'ip_os_day_hour_count',
                    'ip_app_day_hour_count',
                    'ip_app_os_day_hour_count',
                    'app_day_hour_count',
                    'ip_device_os_count',
                    'ip_app_device_os_count']

feature_mean = ['ip_device_os_mean',
                'ip_app_device_os_mean', 'ip_app_device_mean', 'app_device_os_mean']

# feature_reversemean = ['ip_device_os_reversemean',
#                 'ip_app_device_os_reversemean', 'ip_reversemean']
feature_reversemean = []

feature_time2nextclick = ['ip_device_os_time2nextclick',
                            'ip_app_device_os_time2nextclick', 'ip_app_device_time2nextclick', 'app_device_os_time2nextclick']

feature_time2previousclick = ['ip_device_os_time2previousclick', 
                                'ip_app_device_os_time2previousclick', 'ip_app_device_time2previousclick', 'app_device_os_time2previousclick']
    
    
feature_countfromfuture = ['ip_device_os_countfromfuture',
                            'ip_app_device_os_countfromfuture', 'ip_app_device_countfromfuture', 'app_device_os_countfromfuture']

feature_countfrompast = ['ip_device_os_countfrompast',
                            'ip_app_device_os_countfrompast', 'ip_app_device_countfrompast', 'app_device_os_countfrompast']
    
feature_lasttimediff =  ['ip_device_os_lasttimediff',
                             'ip_app_device_os_lasttimediff', 'ip_app_device_lasttimediff', 'app_device_os_lasttimediff']

# feature_matrixfac = ['matrixFact_user_iposdeviceapp_item_device', 
#                      'matrixFact_user_iposdeviceapp_item_app','matrixFact_user_iposdeviceapp_item_channel']
feature_matrixfac = [ 'matrixFact_user_iposdeviceapp_item_app', 'matrixFact_user_ip_item_appdeviceos']



feature_var = ['ip_app_os_var_hour', 'ip_app_channel_var_day']
feature_var = [] # best result need to add var
feature_regression = ['ip_device_os_regression', 'ip_app_device_os_regression', 'ip_app_device_regression', 'app_device_os_regression']
                         

feature_ori = ['app', 'channel', 'device', 'os', 'hour']

feature_cols = []
added_feature = []

added_feature.extend(feature_count)
added_feature.extend(feature_mean)
added_feature.extend(feature_reversemean)
added_feature.extend(feature_time2nextclick)
added_feature.extend(feature_time2previousclick)
added_feature.extend(feature_countfromfuture)
added_feature.extend(feature_countfrompast)
added_feature.extend(feature_lasttimediff)
added_feature.extend(feature_matrixfac)
added_feature.extend(feature_var)
added_feature.extend(feature_regression)
feature_cols.extend(added_feature)
feature_cols.extend(feature_ori)

train_cols = feature_cols.copy()
train_cols.append(target)

In [6]:
feature_cols

['ip_day_hour_count',
 'ip_os_day_hour_count',
 'ip_app_day_hour_count',
 'ip_app_os_day_hour_count',
 'app_day_hour_count',
 'ip_device_os_count',
 'ip_app_device_os_count',
 'ip_device_os_mean',
 'ip_app_device_os_mean',
 'ip_app_device_mean',
 'app_device_os_mean',
 'ip_device_os_time2nextclick',
 'ip_app_device_os_time2nextclick',
 'ip_app_device_time2nextclick',
 'app_device_os_time2nextclick',
 'ip_device_os_time2previousclick',
 'ip_app_device_os_time2previousclick',
 'ip_app_device_time2previousclick',
 'app_device_os_time2previousclick',
 'ip_device_os_countfromfuture',
 'ip_app_device_os_countfromfuture',
 'ip_app_device_countfromfuture',
 'app_device_os_countfromfuture',
 'ip_device_os_countfrompast',
 'ip_app_device_os_countfrompast',
 'ip_app_device_countfrompast',
 'app_device_os_countfrompast',
 'ip_device_os_lasttimediff',
 'ip_app_device_os_lasttimediff',
 'ip_app_device_lasttimediff',
 'app_device_os_lasttimediff',
 'matrixFact_user_iposdeviceapp_item_app',
 'matrixFa

# Define functin

In [7]:
def count(df_history, df_train, cols, target=None):
    """
    Purpose: add a new feature to training df.count the number of records for each feature combination (eg, artist_name_composer)) 
    """
    
    group = get_group(df_train, cols)
    group_all = get_group(df_history, cols)
    
    count_map = group_all.value_counts()
    
    return group.map(count_map).fillna(0)

def countsort(df_history, df_train, cols, target=None):
    """
    Purpose: add a new feature to training df.count the number of records for each feature combination (eg, artist_name_composer)) 
    """
    
    group = get_group(df_train, cols)
    group_all = get_group(df_history, cols)
    
    count_map = group_all.value_counts().iloc[::-1]
    count_map.iloc[:] = list(range(1, len(count_map) + 1))
    
    return group.map(count_map).fillna(-1)



def mean(df_history, df_train, cols, target):
    """
    Purpose: add a new feature to training df. conditional probability P(replay (target) | feature combination (eg, artist_name_composer)) 
    """
  

    group = get_group(df_train, cols)
    group_history = get_group(df_history, cols)
    mean_map = df_history.groupby(group_history)[target].mean()
    return group.map(mean_map).fillna(-0.01)


def reversemean(df_history, df_train, cols, target):
    """
    Purpose: add a new feature to training df. conditional probability P(replay (target) | feature combination (eg, artist_name_composer)) 
    """
  
    # encoding df's cols into a new series
    group = get_group(df_train, cols)
    # encoding df_history's cols into a new series
    group_history = get_group(df_history, cols)
    # get the conditional probability p(target| feature combination. eg, artist_name_composer) 
    positive = group_history[df_history[target] == 1]
    negative = group_history[df_history[target] == 0]
    index_p = set(positive.unique())
    index_n = set(negative.unique())
    index_n.difference_update(index_p)
    map_reverse_p = positive.groupby(positive).count() / len(positive)
    map_reverse_n = pd.Series(np.zeros(len(index_n)), index=index_n)
    map_reverse = pd.concat([map_reverse_p, map_reverse_n])
    return group.map(map_reverse).fillna(-1)


def time2nextclick(df_history, df_train, cols, target, timecol='timestamp'):
    
    result = []
    df_reverse = df_train.sort_index(ascending=False)
    group = get_group(df_reverse,  cols)
    
    next_heard = {}
    for g, t in zip(group, df_reverse[timecol]):
        if g in next_heard:
            result.append(next_heard[g] - t)
        else:
            result.append(-1)
        next_heard[g] = t
    
    result.reverse()
    return result

def time2previousclick(df_history, df_train, cols, target, timecol='timestamp'):
    
    result = []
    group = get_group(df_train, cols)

    last_heard = {}
    for t, g in zip(df_train[timecol], group):
        if g in last_heard:
            result.append(t - last_heard[g])
        else:
            result.append(-1)
        last_heard[g] = t
        
    return result

def countfrompast(df_history, df_train, cols, target, timecol='timestamp'):
    
    group = get_group(df_train, cols)
    
    count = {}
    result = []
    for g in group.values:
        if g not in count:
            count[g] = 0
        else:
            count[g] += 1
        result.append(count[g])
        
    return result

def countfromfuture(df_history, df_train, cols, target, timecol='timestamp'):
    
    result = []
    df_reverse = df_train.sort_index(ascending=False)
    group = get_group(df_reverse,  cols)
    
    count = {}
    for g in group.values:
        if g in count:
            result.append(count[g])
            count[g] += 1 
        else:
            result.append(0)
            count[g] = 1
    
    result.reverse()
    return result

def lasttimediff(df_history, df_train, cols, target, timecol='timestamp'):
    
    group = get_group(df_train, cols)
        
    last_time = df_train.groupby(group)[timecol].last()
    
    return group.map(last_time) - df_train[timecol]

def col_name(cols, func=None):
    if func is None:
        return '_'.join(cols)
    else:
        return '_'.join(cols) + '_' + func.__name__
    
    
from lightfm import LightFM
import scipy.sparse as sp
from scipy.sparse import coo_matrix
from lightfm import LightFM
from sklearn.preprocessing import LabelEncoder

def get_var(df_history, df, group_col, agg_col):
    group = get_group(df, group_col)
    group_history = get_group(df_history, group_col)
    df_temp = pd.DataFrame()
    df_temp['group'] = group_history.values
    df_temp['agg'] = df_history[agg_col].values
    group_map =df_temp.groupby('group')['agg'].var()
    result = group.map(group_map).fillna(0)
    return result

def matrix_factorization(df_history, df, target, item_col, userid_col, userraw_col):
    """
    userid_col is unique user id
    item_col is unique itme id
    userraw_col is used to construct user feature. dim: user_id*userraw
    """
    dff = pd.DataFrame()
    dff_history = pd.DataFrame()


    #1. process item
    if item_col is None:
        dff['item'] = np.zeros(len(df))
        dff_history['item'] = np.zeros(len(df_history))
    else:
        encoder = LabelEncoder()
        group = get_group(df, item_col)
        group_history = get_group(df_history, item_col)
        encoder.fit(pd.concat([group, group_history]))
        dff['item'] = encoder.transform(group)
        dff_history['item'] = encoder.transform(group_history)
#     print('processing item done!')

    #2. user raw
    group = get_group(df, userraw_col)
    group_history = get_group(df_history, userraw_col)
    encoder = LabelEncoder()
    encoder.fit(pd.concat([group, group_history]))
    dff['userraw'] = encoder.transform(group)
    dff_history['userraw'] = encoder.transform(group_history)
#     print('processing user raw done')


    #3. user_id
    group = get_group(df, userid_col)
    group_history = get_group(df_history, userid_col)
    encoder = LabelEncoder()
    encoder.fit(pd.concat([group, group_history]))
    dff['user_id'] = encoder.transform(group)
    dff_history['user_id'] = encoder.transform(group_history)
#     print('processing user id done')



    num_users = max(dff.user_id.max(), dff_history.user_id.max()) + 1
    num_items = max(dff.item.max(), dff_history.item.max()) + 1
    num_userraw = max(dff.userraw.max(), dff_history.userraw.max()) + 1

    M = coo_matrix(
            (df_history[target], ( dff_history.user_id, dff_history.item)),
            shape=(num_users, num_items)
        )

    user_features = pd.concat([dff, dff_history])[['userraw', 'user_id']].drop_duplicates()

    user_features = coo_matrix(
        (np.ones(len(user_features)), (user_features.user_id, user_features.userraw)),
        shape=(num_users, num_userraw)
    )

    user_features = sp.hstack([sp.eye(num_users), user_features])

    model = LightFM(no_components=50, learning_rate=0.1)
    print('fitting lightFM')
    model.fit(
            M, 
            epochs=2, 
            num_threads=36, 
            user_features=user_features,
        )
    print('predicting lightFM')
    result = model.predict(
        dff.user_id.values, 
        dff.item.values, 
        user_features=user_features,
    )
    return result



def regression(df_history, df, cols, target= 'is_attributed', time_col='timestamp', shift=1500000000):
    df = df.copy()
    df_history = df_history.copy()
    df.loc[:,time_col] = df.loc[:,time_col] - shift
    df_history.loc[:,time_col] = df_history.loc[:,time_col] - shift
    group = get_group(df, cols)
    group_history = get_group(df_history, cols)

    targets = {}
    times = {}
    for (y, t), u in zip(df_history[[target, time_col]].values, group_history):
        if u not in targets:
            targets[u] = [y]
            times[u] = [t]
        else:
            targets[u].append(y)
            times[u].append(t)

    linal_user = {}
    for u in times:
        if len(times[u]) > 1:
            A = np.vstack([times[u], np.ones(len(times[u]))]).T
            linal_user[u] = np.linalg.inv(A.T.dot(A)).dot(A.T).dot(targets[u])

    result = []

    for t, u in zip(df[time_col], group):
        if u not in times:
            result.append(-0.5)
        else:
            if len(times[u]) < 2:
                result.append(-0.5)
            else:
                result.append(linal_user[u].dot([t, 1]))
    return result
    


In [8]:
orders = {}
feature_col = ['ip', 
              'app', 
              'device', 
              'os', 
              'channel',
              'day',
              'hour',]

# feature_col = ['ip', 
#               'app', 
#               'device', 
#               'os', 
#               'channel']
for col in feature_col:
    orders[col] = 10 ** (int(np.log(max(train[col].max(),test[col].max() ) + 1) / np.log(10)) + 1)
def get_group(df, cols):
    """
    define an encoding method which can ganrantee the adding value will be unique.
    eg: artist_name_composer will be a combination of (artist_name,composer) and the encoding will reflect the unqiue combination of those two
    """
    group = df[cols[0]].copy()
    for col in cols[1:]:
        group = group * orders[col] + df[col]
        
    return group

# Feature engineering on Train

In [9]:
import sys
from itertools import combinations
combine_col = ['ip', 
              'app', 
              'device', 
              'os', 
              'channel',
              'day',
              'hour',]


counter = 0
df_train = df_comp.copy()
print('got train data')
history_index = list(set(train.index.values) - set(df_train.index.values))
df_history = train.iloc[history_index].copy()
print('got historical data')

###########################################################################
for func in [count, mean, reversemean,time2nextclick, time2previousclick, countfromfuture, countfrompast, lasttimediff, regression]:
            if func.__name__ == count.__name__:
                df_all = pd.concat([train, test])
            else:
                try:
                    del df_all
                    gc.collect()
                except Exception:
                    print('df_all does not exist')

            for num_col in [1,2,3,4,5]:
                for cols in combinations(combine_col, num_col):
                    feature_name = col_name(cols, func=func)
                    if feature_name not in added_feature:
                           continue
                    counter += 1
                    if func.__name__ == count.__name__:
                            print('count function')
                            df_train[feature_name] = func(df_all, df_train, cols, target='is_attributed')

                    elif func.__name__ == mean.__name__:
                            print('mean function')
                            df_train[feature_name] = func(df_history, df_train, cols, target='is_attributed')
                    elif func.__name__ == reversemean.__name__:
                            print('reverse mean function')
                            df_train[feature_name] = func(df_history, df_train, cols, target='is_attributed')

                    elif func.__name__ == regression.__name__:
                            print('regression function')
                            df_train[feature_name] = np.ones(len(df_train)) * (-0.5)

                    else:
                            print('time related function')
                            df_train[feature_name] = np.zeros(len(df_train))

                    all_str = 'all {}:   {}   \t\t\t size: {} G.'.format(counter, feature_name, sys.getsizeof(df_train)/ 1024 **3)
                    print(all_str)
                    with open('feature_all.txt', 'w') as text_file:
                        text_file.write(all_str + '\n')

    
    
#     ### get val
#     print('get val')
#     df_all = pd.concat([train, test])
#     df_train['ip_app_os_var_hour'] = get_var(df_all, df_train, ['ip','app', 'os'], 'hour')
#     print('ip_app_os_var_hour done!')
#     df_train['ip_app_channel_var_day'] = get_var(df_all, df_train, ['ip','app', 'channel'], 'day')
#     print('ip_app_channel_var_day done!')
#     del df_all
#     gc.collect()
    
    
### matrix - factorization
counter += 1
feature_name = 'matrixFact_user_iposdeviceapp_item_app'
print('processing {}'.format(feature_name))
df_train[feature_name] = matrix_factorization(df_history, df_train,target, item_col=['app'], userid_col=['ip','os','device','app'], userraw_col=['ip'])
feature_name = 'matrixFact_user_ip_item_appdeviceos'
print('processing {}'.format(feature_name))
counter += 1
df_train[feature_name] = matrix_factorization(df_history, df_train,target, item_col=['app', 'device', 'os'], userid_col=['ip'], userraw_col=['ip'])


all_str = 'all {}:   {}   \t\t\t size: {} G.'.format(counter, feature_name, sys.getsizeof(df_train)/ 1024 **3)
print(all_str)
with open('feature_all.txt', 'w') as text_file:
    text_file.write(all_str + '\n')


save_file_name = '{}_features_matrixregV2_comp.csv'.format('Train')
save_file_path = '/home/kai/data/kaggle/talkingdata/wl/data/equalhour/' + save_file_name
df_train = df_train[train_cols]
print('------')
print(df_train.columns.values)
df_train.to_csv(save_file_path, index=False)
print(save_file_path)
print('======================================================')

got train data
got historical data
count function
all 1:   ip_device_os_count   			 size: 8.555501699447632e-05 G.
count function
all 2:   ip_day_hour_count   			 size: 9.166449308395386e-05 G.
count function
all 3:   app_day_hour_count   			 size: 9.77739691734314e-05 G.
count function
all 4:   ip_app_device_os_count   			 size: 0.00010388344526290894 G.
count function
all 5:   ip_app_day_hour_count   			 size: 0.00010999292135238647 G.
count function
all 6:   ip_os_day_hour_count   			 size: 0.00011610239744186401 G.
count function
all 7:   ip_app_os_day_hour_count   			 size: 0.00012221187353134155 G.
mean function
all 8:   ip_app_device_mean   			 size: 0.0001283213496208191 G.
mean function
all 9:   ip_device_os_mean   			 size: 0.00013443082571029663 G.
mean function
all 10:   app_device_os_mean   			 size: 0.00014054030179977417 G.
mean function
all 11:   ip_app_device_os_mean   			 size: 0.0001466497778892517 G.
df_all does not exist
df_all does not exist
time related function
